In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
def list_files_in_directory(path):
    files = [ f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) ]
    return files

In [ ]:
path_directory = '/home/hwkang/jupyter/root/result/exp10/perf'

In [ ]:
files_list = list_files_in_directory(path_directory)

In [ ]:
files_list.sort()

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df

In [ ]:
def show_boxplot(path_directory, files_list, file_index_start, target_column, title, show=True, save=False, save_path=None):
    df_delta = pd.read_csv(os.path.join(path_directory, files_list[file_index_start+0]))
    df_ttfs = pd.read_csv(os.path.join(path_directory, files_list[file_index_start+1]))
    df_stime = pd.read_csv(os.path.join(path_directory, files_list[file_index_start+2]))
    df_srate = pd.read_csv(os.path.join(path_directory, files_list[file_index_start+3]))

    # 데이터프레임 목록과 이름
    dataframes = {'Delta': df_delta, 'TTFS': df_ttfs, 'S. time': df_stime, 'S. rate': df_srate}

    # 특정 열에 대한 박스 플롯
    column_to_plot = target_column  # 박스 플롯으로 시각화할 열
    filtered_dataframes = {name: remove_outliers(df, column_to_plot) for name, df in dataframes.items()}

    # 박스 플롯을 위한 데이터 준비
    data_to_plot = [df[column_to_plot] for df in filtered_dataframes.values()]

    # 그래프 그리기
    plt.figure(figsize=(6, 6))
    plt.boxplot(data_to_plot, labels=filtered_dataframes.keys())

    # 축과 레이블 추가
    plt.xlabel('Encoding type')
    if( target_column == 'duration' ):
        plt.ylabel(column_to_plot+' (us)')
    elif( target_column == 'pkg' ):
        plt.ylabel('CPU energy consumption (uJ)')
    else:
        plt.ylabel('DRAM energy consumption (uJ)')
    plt.title(title)
    plt.grid(True)

    if save:
        plt.savefig(save_path)
    
    # 그래프 출력
    if show:
        plt.show()

In [ ]:
save_paths = [
    os.path.join(path_directory,'Input_1_[28x28-50-1000].png'),
    os.path.join(path_directory,'Input_2_[32x32-50-1000].png'),
    os.path.join(path_directory,'Input_3_[256x144-50-1000].png'),
    os.path.join(path_directory,'Input_4_[640x480-50-1000].png'),
    os.path.join(path_directory,'Step_1_[32x32-25-1000].png'),
    os.path.join(path_directory,'Step_2_[32x32-50-1000].png'),
    os.path.join(path_directory,'Step_3_[32x32-100-1000].png'),
    os.path.join(path_directory,'Step_4_[32x32-1000-1000].png'),
    os.path.join(path_directory,'Iter_1_[32x32-50-1000].png'),
    os.path.join(path_directory,'Iter_2_[32x32-50-2000].png'),
    os.path.join(path_directory,'Iter_3_[32x32-50-4000].png'),
    os.path.join(path_directory,'Iter_4_[32x32-50-8000].png')
]

titles = [
    '[Input_size, Num_step, Num_iter]: [28x28, 50, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 50, 1000]',
    '[Input_size, Num_step, Num_iter]: [256x144, 50, 1000]',
    '[Input_size, Num_step, Num_iter]: [640x480, 50, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 25, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 50, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 100, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 1000, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 50, 1000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 50, 2000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 50, 4000]',
    '[Input_size, Num_step, Num_iter]: [32x32, 50, 8000]',
]

In [ ]:
for i in range(12):
    show_boxplot(path_directory, files_list, file_index_start=i*4, target_column='duration', title=titles[i], show=False, save=True, save_path=save_paths[i])

In [ ]:
for i in range(12):
    show_boxplot(path_directory, files_list, file_index_start=i*4, target_column='pkg', title=titles[i], show=False, save=True, save_path=save_paths[i])

In [ ]:
for i in range(12):
    show_boxplot(path_directory, files_list, file_index_start=i*4, target_column='dram', title=titles[i], show=False, save=True, save_path=save_paths[i])